<a href="https://colab.research.google.com/github/archie-a18/zomato/blob/main/Copy_of_Untitled8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Colab: Install dependencies & Gradio
!apt-get update -y && apt-get install -y tesseract-ocr
!pip install --quiet \
    torch torchvision torchaudio \
    transformers sentence-transformers faiss-cpu \
    pytesseract Pillow requests scikit-learn gradio


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,844 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,118 kB]
Fetched 7,219 kB in 4s (1,733 kB/s)
Reading package lists... Done
W: Skipping acquire of co

In [11]:
import re, requests, gradio as gr
from io import BytesIO
from PIL import Image
import pytesseract
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# ──────────────── Static menu URLs ────────────────
RESTAURANT_MENUS = {
    "Pukhtaan": [
        "https://b.zmtcdn.com/data/menus/853/21580853/feb85f6fa20259fe481a7bc440c24476.jpg"
    ],
    "Connaught_Club_House": [
        "https://b.zmtcdn.com/data/menus/106/19295106/faf36abd62cb22e25492a1e51d36d971.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/6d533623bfb316c927d5087a1be26f1b.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/92fdde9055013c6ab9ba7e9b76c1770d.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/d59c8c08538a220438fab471790d2b3b.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/d5ff7bf34e11499d79ee59b5492e892d.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/5757370e09010293de96576c65c499f3.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/e784555ca0fac7f4b2c78ad12d708108.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/cf56b2e45236ab81e3adeebde09937e5.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/b3a95e356d17115f48f31e488fa14510.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/db6351e453b6a553e2a95a029ccd5c93.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/63668b33e2cd94b084aba2400a32302d.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/28c47c6101f4b6525ae0f00054685f7b.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/8580a3bc941162b7ac0fcf6a2b29408f.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/9e199b47a96c620137325a2edaa257ea.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/a48b905eda7e351ec9d16be38263c6ce.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/e1256a12cfc8d2671887daf5849462c3.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/407cf6d0c92226a1e8d3c2e306bc7eee.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/a06bc4526e30e16cd66e7fc33680ddd1.jpg",
    "https://b.zmtcdn.com/data/menus/106/19295106/f43d42f35215a54255c2b3c5e7bc361d.jpg"
],
    "Local": [
   "https://b.zmtcdn.com/data/menus/360/18382360/4a261b83c8d83e45c90ba18738606383.jpg",
    "https://b.zmtcdn.com/data/menus/360/18382360/e594cde054e3b9cd74bada9713d45647.jpg",
    "https://b.zmtcdn.com/data/menus/360/18382360/09f07621c6c933873eaa0e3cc2d8cee6.jpg",
    "https://b.zmtcdn.com/data/menus/360/18382360/9df0e7ea170f93df343a776cbb433b2e.jpg",
    "https://b.zmtcdn.com/data/menus/360/18382360/c7a32dfe8174158daf3395a2d8a3122a.jpg",
    "https://b.zmtcdn.com/data/menus/360/18382360/6213b5c0e0e6304b496aa40ec29ee7c0.jpg",
    "https://b.zmtcdn.com/data/menus/360/18382360/62bb0f0ab165aed9aac2f07386be6ce0.jpg",
    "https://b.zmtcdn.com/data/menus/360/18382360/9bb62a9813fd95783befe1f938dff819.jpg",
    "https://b.zmtcdn.com/data/menus/360/18382360/39c8d9378a98634edad40bd2e21c8269.jpg",
    "https://b.zmtcdn.com/data/menus/360/18382360/9059e84501429b654a3b187f8fa05f3b.jpg"
],
}


def extract_text(u):
    r = requests.get(u, timeout=10); r.raise_for_status()
    return pytesseract.image_to_string(Image.open(BytesIO(r.content)))

def clean_txt(txt):
    txt = re.sub(r'[^A-Za-z0-9\s\.,₹$€¥¢&+()/-]', '', txt)
    fixes = {
        r'\b(O|0)e\b':'Of',
        r'\b[Tt]andoori\s+Alo\b':'Tandoori Aloo',
        r'\bPaneera?\b':'Paneer',
        r'\bMurgh\b':'Chicken',
        r'\bCCH\b':'Connaught Club House',
        r'\bR\s*(\d+)':r'₹\1',
        r'\b(\d{1,3})\s*-\s*(\d{1,3})\b':r'₹\1-₹\2'
    }
    for p,rp in fixes.items():
        txt = re.sub(p, rp, txt)
    return txt

# ─── 3) Parse & classify ──────────────────────────────────────────────────────
def parse_items(raw):
    lines = [l.strip() for l in clean_txt(raw).splitlines() if l.strip()]
    pr = re.compile(r'(₹|Rs\.?|INR)\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)', re.I)
    items, cur = [], None
    for ln in lines:
        m = pr.search(ln)
        if m and cur:
            cur['price'] = m.group().strip()
            items.append(cur)
            cur = None
            continue
        if ln[0].isupper() and len(ln)>3:
            if cur: items.append(cur)
            cur = {'name':ln, 'price':'', 'description':''}
        elif cur:
            cur['description'] += ' ' + ln
    if cur: items.append(cur)
    return items

def classify(items):
    diet_cfg = {
        'non-vegetarian':['chicken','mutton','fish','prawn','meat','lamb','beef','egg','shrimp','crab'],
        'vegetarian':['veg','paneer','dal','sabzi','salad','tofu','mushroom','potato'],
        'gluten-free':['gluten-free','gluten free','gf'],
        'vegan':['vegan','dairy-free','plant-based']
    }
    spice_cfg = {
        3:['volcano','ghost pepper','extra spicy','fiery'],
        2:['spicy','chilli','hot','jalapeno'],
        1:['mild','paprika','light spice']
    }
    for it in items:
        txt = (it['name'] + ' ' + it['description']).lower()
        tags=[]
        if any(w in txt for w in diet_cfg['non-vegetarian']): tags.append('non-vegetarian')
        if any(w in txt for w in diet_cfg['vegetarian']):     tags.append('vegetarian')
        for t in ('gluten-free','vegan'):
            if any(w in txt for w in diet_cfg[t]): tags.append(t)
        it['dietary']=tags or ['unspecified']
        it['is_veg']=('non-vegetarian' not in tags)
        lvl=0
        for lv,kws in spice_cfg.items():
            if any(w in txt for w in kws): lvl=max(lvl,lv)
        it['spice_level']=lvl
    return items

# ─── 4) Build restaurant_data ─────────────────────────────────────────────────
def build_data():
    D={}
    for r,urls in RESTAURANT_MENUS.items():
        pages = [extract_text(u) for u in urls]
        its = parse_items("\n\n".join(pages))
        D[r] = {'items': classify(its)}
    return D

restaurant_data = build_data()

# ─── 5) Embed + FAISS ──────────────────────────────────────────────────────────
docs=[]
for r,info in restaurant_data.items():
    for it in info['items']:
        docs.append(f"{it['name']} — {it['description']} [{r}]")

embedder = SentenceTransformer('all-MiniLM-L6-v2')
embs     = embedder.encode(docs, convert_to_numpy=True, show_progress_bar=True)
index    = faiss.IndexFlatL2(embs.shape[1]); index.add(embs)

# ─── 6) RAG chain setup ────────────────────────────────────────────────────────
tok   = AutoTokenizer.from_pretrained("google/flan-t5-small")
mdl   = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
pipe  = pipeline("text2text-generation", model=mdl, tokenizer=tok, max_length=512)

def rag_answer(q, k=5):
    q_emb = embedder.encode([q], convert_to_numpy=True)
    D,I   = index.search(q_emb, k)
    ctx   = "\n".join(f"- {docs[i]}" for i in I[0])
    prompt= f"You are a menu expert.\nMenu Items:\n{ctx}\n\nQuestion: {q}\nAnswer:"
    return pipe(prompt, max_length=512, do_sample=False)[0]['generated_text'].strip()

# ─── 7) Template handlers ──────────────────────────────────────────────────────
def best_veg():
    ratios={r:sum(it['is_veg'] for it in v['items'])/len(v['items']) for r,v in restaurant_data.items()}
    b=max(ratios,key=ratios.get)
    return f"🏆 **{b}** has the best vegetarian menu: {ratios[b]*100:.1f}% veg items."

def gf_apps(r):
    R=r.title()
    if R not in restaurant_data: return f"❌ '{R}' not found."
    gf=[it for it in restaurant_data[R]['items'] if 'gluten-free' in it['dietary']]
    if not gf: return f"❌ No gluten-free items at {R}."
    return "**Gluten-Free Appetizers @ "+R+"**\n" + "\n".join(f"- {it['name']} — {it['price']}" for it in gf)

def dessert_range(r):
    R = r.title()
    if R not in restaurant_data:
        return f"❌ '{R}' not found."

    dessert_keywords = [
        'dessert', 'ice cream', 'kulfi', 'gulab jamun', 'rasgulla', 'rasmalai',
        'brownie', 'cake', 'cheesecake', 'pastry', 'halwa', 'pudding', 'phirni',
        'laddu', 'sweet', 'jalebi'
    ]

    des = [it for it in restaurant_data[R]['items'] if any(kw in (it['name'] + ' ' + it['description']).lower() for kw in dessert_keywords)]

    if not des:
        return f"❌ No desserts found at {R}."

    prices = []
    for it in des:
        digits = re.findall(r'\d+', it['price'])
        if digits:
            prices.append(int(digits[0]))

    if not prices:
        return f"ℹ️ Desserts found but no clear prices listed at {R}."

    return f"🍰 **Dessert price range @ {R}**: ₹{min(prices)} - ₹{max(prices)}"

def paneer_price_range(restaurant_name):
    if restaurant_name not in restaurant_data:
        return f"No data found for {restaurant_name}"

    prices = []
    for item in restaurant_data[restaurant_name]['items']:
        name_lower = item['name'].lower()
        desc_lower = item['description'].lower()
        if 'paneer' in name_lower or 'paneer' in desc_lower:
            price_matches = re.findall(r'\d+', item['price'])
            if price_matches:
                prices.extend([int(p) for p in price_matches])

    if prices:
        return f"📍 {restaurant_name}\nPaneer items price range: ₹{min(prices)} – ₹{max(prices)}"
    else:
        return f"No paneer items with prices found for {restaurant_name}"




def compare_spice(a,b):
    def avg(r): lv=[it['spice_level'] for it in restaurant_data.get(r,{}).get('items',[])]; return sum(lv)/len(lv) if lv else 0
    a1,a2=avg(a),avg(b)
    w= a if a1>a2 else b if a2>a1 else "Both"
    return f"🌶️ {a}: {a1:.2f}/3   {b}: {a2:.2f}/3   → Higher at **{w}**."

# ─── 8) Formatter ─────────────────────────────────────────────────────────────
def format_grouped(res):
    if not res: return "😕 No matches found."
    out=""
    by_rest={}
    for it in res: by_rest.setdefault(it['restaurant'],[]).append(it)
    for r,its in by_rest.items():
        out+=f"### 📍 {r}\n"
        for it in its:
            diet = "🌱" if it['is_veg'] else "🍖"
            if 'gluten-free' in it['dietary']: diet+="🌾"
            if 'vegan' in it['dietary']:      diet+="🌿"
            spice = "🌶️"*it['spice_level'] or "🟢 Mild"
            out+=f"- **{it['name']}** — {it['price']}   {spice}   [{diet}]\n"
        out+="\n"
    return out

# ─── 9) Chatbot logic ─────────────────────────────────────────────────────────
def chatbot(q):
    low=q.lower()
    # templates
    if "best vegetarian" in low:                   return best_veg()
    m=re.search(r"does\s+(.+?)\s+have.*gluten[- ]free", low)
    if m:                                         return gf_apps(m.group(1))
    m=re.search(r"price range.*?(\w+)'?s dessert", low)
    if m:                                         return dessert_range(m.group(1))
    m=re.search(r"compare.*spice.*menus of (.+?) and (.+)", low)
    if m:                                         return compare_spice(m.group(1).title(), m.group(2).title())
    # attribute filters
    res=[]
    if "veg" in low and "non" not in low:
        for r,v in restaurant_data.items():
            for it in v['items']:
                if it['is_veg']: res.append({**it,'restaurant':r})
        return format_grouped(res)
    if any(x in low for x in ("non-veg","non veg")):
        for r,v in restaurant_data.items():
            for it in v['items']:
                if not it['is_veg']: res.append({**it,'restaurant':r})
        return format_grouped(res)
    if "gluten" in low:
        for r,v in restaurant_data.items():
            for it in v['items']:
                if 'gluten-free' in it['dietary']: res.append({**it,'restaurant':r})
        return format_grouped(res)
    m=re.search(r'under ₹?(\d+)', q)
    if m:
        lim=int(m.group(1))
        for r,v in restaurant_data.items():
            for it in v['items']:
                p=int(re.sub(r'[^0-9]','', it['price'] or '0') or 0)
                if p and p<=lim: res.append({**it,'restaurant':r})
        return format_grouped(res)
    if ' in ' in low:
        rest=low.split(' in ')[-1].title()
        if rest in restaurant_data:
            for it in restaurant_data[rest]['items']:
                res.append({**it,'restaurant':rest})
            return format_grouped(res)
    if any(x in low for x in ("spicy","hot","chilli")):
        for r,v in restaurant_data.items():
            for it in v['items']:
                if it['spice_level']>0: res.append({**it,'restaurant':r})
        return format_grouped(res)
    # ingredient search
    terms=re.findall(r"\b\w{4,}\b", low)
    for r,v in restaurant_data.items():
        for it in v['items']:
            blob=(it['name']+' '+it['description']).lower()
            if any(t in blob for t in terms): res.append({**it,'restaurant':r})
    if res: return format_grouped(res)
    # fallback
    return rag_answer(q)



Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Device set to use cpu


In [12]:
gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask anything…"),
    outputs="markdown",
    title="🍴 Food Explorer RAG Chatbot",
    description="Dietary, price, spice, comparisons, or any free‐text query",
    allow_flagging="never"
).launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6208c418fa6d14d6d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
